In [102]:
import pandas as pd
import numpy as np
import scipy.stats
import random
from sklearn import preprocessing

# geo
from pygeotile.tile import Tile


import imp
import plot_maps as pm
imp.reload(pm)
%matplotlib inline

In [103]:
# функция выбирает те терминалы, которые могут иметь несколько координат на карте, но расположены недалеко друг от друга
# по умолчанию оставим значение ~ равное "терминал мог изменить своё местоположение внутри района - города"
def short_distance_terminals(df, threshold = 0.1):
    min_coords_lat = df.groupby(['terminal_id'])['tran_lat'].min().reset_index(name='min_lat')
    max_coords_lat = df.groupby(['terminal_id'])['tran_lat'].max().reset_index(name='max_lat')
    min_coords_lon = df.groupby(['terminal_id'])['tran_lon'].min().reset_index(name='min_lon')
    max_coords_lon = df.groupby(['terminal_id'])['tran_lon'].max().reset_index(name='max_lon') 
    
    coords_diff = max_coords_lat
    coords_diff['min_lat'] = min_coords_lat['min_lat'] 
    coords_diff['min_lon'] = min_coords_lon['min_lon']
    coords_diff['max_lon'] = max_coords_lon['max_lon']  
    coords_diff['diff_lat'] = abs(coords_diff['min_lat'] - coords_diff['max_lat'])
    coords_diff['diff_lon'] = abs(coords_diff['min_lon'] - coords_diff['max_lon'])
    
    good_terminals = coords_diff[(coords_diff.diff_lat < threshold) &
                                 (coords_diff.diff_lon < threshold)]['terminal_id'].unique()
    return good_terminals

# объединяем координаты транзакции  под одни колонки (*_1 может быть нулевой, порядок агрументов важен)
def concat_coords(dataframe, concat_lat_1, concat_lon_1, concat_lat_2, concat_lon_2):
    dataframe['tran_lat'] = np.where(dataframe[concat_lat_1].isnull(),
                                    dataframe[concat_lat_2], dataframe[concat_lat_1])
    dataframe['tran_lon'] = np.where(dataframe[concat_lon_1].isnull(),
                                    dataframe[concat_lon_2], dataframe[concat_lon_1])
    return dataframe

# если у терминала одна координата - оставить. Если много - заменить средним
def change_coords_by_mean(dataframe):
    # число уникальных гео координат на терминал
    terminals_unique_pos = dataframe.groupby('terminal_id')['trans_pos'].nunique().reset_index(name = 'count')
    dual_terminal_pos = terminals_unique_pos[terminals_unique_pos['count'] > 1]['terminal_id']
    
    # оставляем те, что встречаются больше одного раза и заменяем средним
    tmp = dataframe[dataframe['terminal_id'].isin(dual_terminal_pos)]
    terminals_mean = tmp.groupby('terminal_id')[['tran_lat','tran_lon']].apply(np.mean).reset_index()
    
    dataframe = dataframe.merge(terminals_mean, how = 'left', left_on = 'terminal_id', right_on = 'terminal_id')
    dataframe = concat_coords(dataframe, 'tran_lat_y', 'tran_lon_y', 'tran_lat_x', 'tran_lon_x')
    dataframe.drop(['tran_lat_y', 'tran_lon_y', 'tran_lat_x', 'tran_lon_x'], axis = 1, inplace = True)
    del tmp
    return dataframe
    
# предобработка исходных данных. Удаление ненужных столбцов  и плохих терминалов.
def preprosses(dataframe, df_mcc):
    
    num_old_trans = dataframe.shape[0]
    num_old_cust_ids = dataframe['customer_id'].nunique()
    num_old_term_ids = dataframe['terminal_id'].nunique()
    print ('Before data preprossesing\n shape: %d, unique clients: %d, unique terminals: %d' 
                           %(num_old_trans, num_old_cust_ids, num_old_term_ids))
    
    # объединим координаты банкоматов и прочих транзакций, сформировав единое представление о координатах
    dataframe = concat_coords(dataframe, 'atm_address_lat', 'atm_address_lon'
                                       , 'pos_adress_lat','pos_adress_lon')
    
    dataframe = pd.merge(dataframe, df_mcc, left_on = 'mcc', right_on='MCC')
    
    # те mcc коды, которые не смержились, можно убрать
    dataframe = dataframe[~dataframe["mcc_group"].isnull()]
    # кафе и фастфуд будем считать одним и тем же
    dataframe['mcc_group']  = dataframe['mcc_group'].replace({41: 19})  
    
    
    dataframe = dataframe[dataframe['country'].isin(['RUS','RU'])]
    
    # эти колонки больше не нужны
    dataframe.drop(['MCC'
                    ,'pos_adress_lat','pos_adress_lon'
                    ,'atm_address_lat','atm_address_lon'
                    ,'atm_address', 'pos_address'], axis = 1, inplace = True)
   
    # редкие mcc объединим в одно
    tmp = dataframe.groupby('mcc_group')['terminal_id'].nunique().reset_index()
    rare_mcc = tmp[tmp['terminal_id'] < 3000]['mcc_group']
    dataframe.loc[dataframe['mcc_group'].isin(rare_mcc), 'mcc_group'] = 0
    
    # mcc_common -> dummies
    dataframe['mcc_common'] = dataframe['mcc_group']
    dataframe = pd.get_dummies(dataframe, columns=["mcc_group"])
    
    # удаляем те терминалы, которые без географической привязки. По заданному адресу их сложно восстановить.
#     terminals = dataframe[dataframe['mcc_common'] == 22]
#     nan_terminals = terminals[terminals['tran_lat'].isnull()]['terminal_id'].unique()
#     dataframe = dataframe[~dataframe['terminal_id'].isin(nan_terminals)]

    # убираем те терминалы, которые меняли свою привзяку по mcc
#     terminals_count = dataframe.groupby('terminal_id')['mcc_common'].nunique().reset_index(name = 'count')
#     dual_mcc_terminals = terminals_count[terminals_count['count'] > 1]
#     dataframe = dataframe[~dataframe['terminal_id'].isin(dual_mcc_terminals)]
#     dataframe['dual_mcc'] = dataframe['terminal_id'].isin(dual_mcc_terminals).astype(int)  
    
#     # убираем терминалы, которые "двигались" в пространстве
#     good_terminals = short_distance_terminals(dataframe, 0.2)
#     dataframe['dual_coordinates'] = dataframe['terminal_id'].isin(good_terminals).astype(int)
    
    # дни недели
    dataframe['transaction_date'] = pd.to_datetime(dataframe['transaction_date'])
    dataframe['day_of_week'] = dataframe['transaction_date'].dt.dayofweek
    dataframe['is_weekend'] = dataframe['day_of_week'].apply(lambda x: 1 if (x == 6) | (x == 5) else 0)

    # неработающих/бездомных  отправляем в атлантический океан
#     try:
#         dataframe['work_add_lat'].fillna(0.0, inplace = True)
#         dataframe['work_add_lon'].fillna(0.0, inplace = True)
#         dataframe['home_add_lat'].fillna(0.0, inplace = True)
#         dataframe['home_add_lon'].fillna(0.0, inplace = True)
#     except:
#         pass

    # для термиалов, которые имеют много географических значений - заменяем средним.
    
    dict_median = dataframe[dataframe['mcc_common'] == 22].groupby('terminal_id')[['tran_lat','tran_lon']].median()
    dataframe = dataframe.set_index('terminal_id')
    dataframe.update(dict_median)
    dataframe = dataframe.reset_index()
#     dataframe['trans_pos'] = list(zip(dataframe['tran_lat'], dataframe['tran_lon']))
#     dataframe = change_coords_by_mean(dataframe)  
    dataframe['trans_pos'] = list(zip(dataframe['tran_lat'], dataframe['tran_lon']))
    
    print ('After data preprossesing\n shape: %d, unique clients: %d, unique terminals: %d' 
                           %(dataframe.shape[0], dataframe['customer_id'].nunique(), dataframe['terminal_id'].nunique()))    
    return dataframe

### Читаем и предобрабатываем данные

    - удаляем транзакции, совершённые в терминалах, которые меняли свою привязку в mcc
    - удаляем транзакции, совершённые в терминалах-путешественниках (из разных городов или р-ов одного города)
    - для терминалов с большим числом координат (шум) берём среднее
    - удаление ненужных переменных

In [104]:
df_train = pd.read_csv('data/train_set.csv', low_memory=False)
df_test = pd.read_csv('data/test_set.csv', low_memory = False)

In [105]:
# некоторые специфичные функции для обработки тестового датасета
df_test.columns = ['amount', 'atm_address', 'atm_address_lat', 'atm_address_lon',
       'city', 'country', 'currency', 'customer_id', 'mcc', 'pos_address',
       'pos_adress_lat', 'pos_adress_lon', 'terminal_id',
       'transaction_date']

df_test['mcc'] = df_test['mcc'].apply(lambda x: x.replace(',',''))
df_test['mcc'] = pd.to_numeric(df_test['mcc'])

In [106]:
df_mcc = pd.read_excel('data/mcc.xlsx')
df_mcc = df_mcc[['MCC', 'mcc_group']]
df_train_1 = preprosses(df_train, df_mcc)
df_test_1 = preprosses(df_test, df_mcc)

Before data preprossesing
 shape: 1224734, unique clients: 10000, unique terminals: 208383
After data preprossesing
 shape: 1214411, unique clients: 10000, unique terminals: 205325
Before data preprossesing
 shape: 1265470, unique clients: 9997, unique terminals: 213570
After data preprossesing
 shape: 1252195, unique clients: 9997, unique terminals: 209956


In [107]:
df_train_1.mcc_common.value_counts()

10    458714
22    271797
19    194030
16     45191
0      41865
17     41223
6      35480
7      30928
3      30481
5      26365
26     22607
18     15730
Name: mcc_common, dtype: int64

Посмотрим, как повлияла обработка данных на примере одного клиента

In [108]:
df_train = pd.merge(df_train, df_mcc, left_on = 'mcc', right_on='MCC')
df_train['mcc_common'] = df_train["mcc_group"]

In [109]:
pm.plot_one_person('0dc0137d280a2a82d2dc89282450ff1b', df_train)

   home_add_lat  home_add_lon
0        59.851        30.232
   work_add_lat  work_add_lon
0        59.847        30.177


In [110]:
pm.plot_one_person('0dc0137d280a2a82d2dc89282450ff1b', df_train_1)

   home_add_lat  home_add_lon
0        59.851        30.232
   work_add_lat  work_add_lon
0        59.847        30.177


Видим, что транзакции терминалов (банкоматов) усреднились.

### Стратегия решения задачи:
    - мы будем предсказывать близость транзакции к дому или работе (две задачи классификации)
    - отберём таким образом терминалы с самой высокой вероятностью принадлежности к дому/работе
    - посчитаем средние таких терминалов. Это и будет ответом

In [122]:
# посчитаем для каждого терминала в разрезе пользователя, ближе он к дому или работе. 
def target(dataframe, threshold = 0.02):
    dataframe['near_home'] = (np.sqrt((dataframe['tran_lat'] - dataframe['home_add_lat'])**2 +
                                       (dataframe['tran_lon'] - dataframe['home_add_lon'])**2) < threshold).astype(int)
    dataframe['near_work'] = (np.sqrt((dataframe['tran_lat'] - dataframe['work_add_lat'])**2 +
                                       (dataframe['tran_lon'] - dataframe['work_add_lon'])**2) < threshold).astype(int)
    return dataframe 

In [123]:
df_train_1 = target(df_train_1)

### Генерация фичей

Идеи, которые будем реализовывать:
    + разобьём всю карту на небольшие квадраты (по прообразу тайлов в google maps)
    + для каждой транзакции создадим набор фичей: номер тайла зума n (предлположительно n ~ 15-18, для достаточно малых масштабов) и один чуть побольше (вместо города)
    + для каждой транзакции исходя из зума тайла n посчитать, сколько транзакций/терминалов ещё входит в этот тайл (плотность тайла)
    - день недели(dummies)
    - распределение трат mcc кодов для клиента
    - описание района транзакции (osm, overoass api)
    - статистики: средние транзакции клиента по координатам
    - удалённость транзакции клиента от средней транзакции клиента

In [124]:
# создание тайла определённого масштаба (train + test)
def create_zoom_tile(dataframe, zoom):
    list_tiles_zoom = []
    for ix, row in dataframe.iterrows():
        if np.isnan(row['tran_lat']):
            list_tiles_zoom.append(Tile.for_latitude_longitude(0,0,0))
        else:
            list_tiles_zoom.append(Tile.for_latitude_longitude(row['tran_lat'], row['tran_lon'], zoom))     
    dataframe['tile_n' + str(zoom)] = list_tiles_zoom
    le = preprocessing.LabelEncoder().fit(dataframe['tile_n' + str(zoom)])
    dataframe['tile_' +str(zoom) ] = le.transform(dataframe['tile_n' + str(zoom)])
    if (zoom  < 15):
         dataframe.drop('tile_n' + str(zoom), axis = 1, inplace = True)
    return dataframe
    
# создание тайлов для конкретных масштабов
def create_tiles(dataframe, list_zooms):
    for zoom in list_zooms:
        print ('creating for ', str(zoom))
        dataframe = create_zoom_tile(dataframe, zoom)
    
    return dataframe

# число транзакций в тайле
def calculate_num_of_near_trans(dataframe, tile_column):
    num_of_near_trans = dataframe.groupby(tile_column).size().reset_index(name = 'near_' + tile_column + '_trans_cnt')
    dataframe = dataframe.merge(num_of_near_trans, how  = 'left', left_on = tile_column, right_on = tile_column)
    return dataframe

# число уникальных терминалов в тайле
def calculate_num_of_near_terminals(dataframe, tile_column):
    num_of_near_trerminals = dataframe.groupby(tile_column)['terminal_id'].nunique()\
                                                            .reset_index(name = 'near_' + tile_column + '_terminal_cnt')
    dataframe = dataframe.merge(num_of_near_trerminals, how  = 'left', left_on = tile_column, right_on = tile_column)
    return dataframe

# число транзакций в одинаквых координатах
def number_equal_coordinates(dataframe):
    dict_pos_size = dataframe.groupby(["tran_lat", "tran_lon"]).size().to_dict()
    dataframe["equal_terminals_in_pos"] = dataframe['trans_pos'].map(dict_pos_size)
    return dataframe

# средние и медианы координат в разрезе клиента
def statistics_features(dataframe):
    dict_lat_median = dataframe.groupby('customer_id')['tran_lat'].median().to_dict()
    dict_lon_median = dataframe.groupby('customer_id')['tran_lon'].median().to_dict()
    dict_lat_mean = dataframe.groupby('customer_id')['tran_lat'].mean().to_dict()
    dict_lon_mean = dataframe.groupby('customer_id')['tran_lon'].mean().to_dict()
    dict_lat_std = dataframe.groupby('customer_id')['tran_lat'].std().to_dict()
    dict_lon_std = dataframe.groupby('customer_id')['tran_lon'].std().to_dict()
    
    dataframe['client_2_tran_lat_mean'] = dataframe['customer_id'].map(dict_lat_mean)
    dataframe['client_2_tran_lon_mean'] = dataframe['customer_id'].map(dict_lon_mean)
    dataframe['client_2_tran_lat_meadian'] = dataframe['customer_id'].map(dict_lat_median)
    dataframe['client_2_tran_lon_median'] = dataframe['customer_id'].map(dict_lon_median) 
    dataframe['client_2_tran_lat_std'] = dataframe['customer_id'].map(dict_lat_std)
    dataframe['client_2_tran_lon_std'] = dataframe['customer_id'].map(dict_lon_std)     
    return dataframe

# распределение трат по мерчантам на пользователя
def mcc_freq(dataframe):
    # все транзакции клиента
    dict_trans_cnt = dataframe.groupby('customer_id').size().to_dict()
    dataframe['all_trans_cnt'] = dataframe['customer_id'].map(dict_trans_cnt)
    for mcc in dataframe['mcc_common'].unique():
        dict_mcc_size = dataframe[dataframe['mcc_common'] == mcc].groupby('customer_id').size().to_dict()
        dataframe['mcc_common_' + str(mcc) + '_freq'] = dataframe['customer_id'].map(dict_mcc_size) / dataframe['all_trans_cnt']
        dataframe.fillna(0, inplace = True)
    return dataframe

def calculate_mean_dist(lat, lon, lat_list, lon_list):
#     return np.median(np.sqrt((lat_list-lat)**2 + (lon_list-lon)**2))
    return np.nanmedian(np.sqrt((lat_list-lat)**2 + (lon_list-lon)**2))

def calculate_mean_distance(dataframe):
    ldf = dataframe.copy()
    dict_client2lat = dataframe.groupby('customer_id')['tran_lat'].apply(lambda x: np.array(x)).to_dict()
    ldf['customer_lat'] = ldf['customer_id'].map(dict_client2lat)
    dict_client2lon = dataframe.groupby('customer_id')['tran_lon'].apply(lambda x: np.array(x)).to_dict()
    ldf['customer_lon'] = ldf['customer_id'].map(dict_client2lon)
    dataframe['distance2mean'] = ldf[['tran_lat', 'tran_lon', 'customer_lat', 'customer_lon']]\
                                        .apply(lambda x: calculate_mean_dist(x.tran_lat,
                                                                         x.tran_lon,
                                                                         x.customer_lat,
                                                                         x.customer_lon), axis=1)
    return dataframe

In [114]:
df_all = pd.concat([df_train_1, df_test_1])

In [115]:
%%time
# разобьём всю карту на небольшие квадраты (по прообразу тайлов в google maps)
list_tiles = [10, 12, 13, 14, 15, 16]
df_all = create_tiles(df_all, list_tiles)

creating for  10
creating for  12
creating for  13
creating for  14
creating for  15
creating for  16
Wall time: 15min 10s


In [116]:
# у нас приблизительно столько "городов и сёлов"
df_all.tile_10.nunique()

2514

In [117]:
df_train_1 = df_all.iloc[:df_train_1.shape[0], :]
df_test_1 = df_all.iloc[-df_test_1.shape[0]:, :]

In [118]:
%%time
# для каждой транзакции считаем, сколько уникальных терминалов прошло через тайл + общее число транзакций в этом тайле
for zoom_tile in list_tiles[1:]: 
    df_train_1 = calculate_num_of_near_trans(df_train_1, 'tile_' + str(zoom_tile))
    df_train_1 = calculate_num_of_near_terminals(df_train_1, 'tile_' + str(zoom_tile))
    df_test_1 = calculate_num_of_near_trans(df_test_1, 'tile_' + str(zoom_tile))
    df_test_1 = calculate_num_of_near_terminals(df_test_1, 'tile_' + str(zoom_tile))

Wall time: 44.2 s


In [119]:
%%time
df_train_1 = mcc_freq(df_train_1)
df_train_1 = statistics_features(df_train_1)
df_train_1 = calculate_mean_distance(df_train_1)
df_train_1 = number_equal_coordinates(df_train_1)

df_test_1 = mcc_freq(df_test_1)
df_test_1 = statistics_features(df_test_1)
df_test_1 = calculate_mean_distance(df_test_1)
df_test_1 = number_equal_coordinates(df_test_1)

Wall time: 3min 21s


In [120]:
df_train_1.to_csv('data/train_proc.csv', index = None)
df_test_1.to_csv('data/test_proc.csv', index = None)

In [121]:
df_tiles = pd.concat([df_train_1, df_test_1])['tile_16']

In [126]:
df_tiles = pd.DataFrame(df_train_1[['tile_n16','tile_16']].drop_duplicates(),columns = ['tile_n16','tile_16'])
df_tiles['landuse'] = 0
df_tiles.to_csv('data/tiles_train_16.csv', index = None)